In [1]:
import PIL
import cv2
import skimage
import mahotas
import mahotas.demos
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import tkinter as tk
import import_ipynb
from skimage import feature

from tkinter import *
from PIL import ImageFilter
from tkinter import filedialog
from skimage import filters
from torchvision import models
from PIL import Image
from torchvision.models.segmentation import FCN_ResNet101_Weights
from torchvision.models.segmentation import DeepLabV3_ResNet101_Weights
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import simpledialog
from sklearn.decomposition import PCA
from tkinter.simpledialog import askinteger
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image as keras_image
from keras.applications.vgg16 import preprocess_input



def grayscale(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    grayimg= cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    grayimg1= Image.fromarray(grayimg)

    grayimg1= ImageTk.PhotoImage(grayimg1)

#     panelB = Label(image=grayimg1, borderwidth=5, relief="sunken")
#     panelB.image = grayimg1
#     panelB.grid(row= 1, column=4 , rowspan= 13,columnspan= 3, padx=20, pady=20)
    
    image_label = Label(panelB, image=grayimg1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = grayimg1
    image_label.place(x=50, y=100)
        
        
    new_img=grayimg

    return grayimg

def negative(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    neg= 255 - input_image
    neg1= Image.fromarray(neg)

    neg1= ImageTk.PhotoImage(neg1)

    image_label = Label(panelB, image=neg1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = neg1
    image_label.place(x=50, y=100)

    new_img=neg

    return neg

def threshold(input_image, panelB):   #  image = grayscale()
    global new_img, image_label
    
    threshold_value = simpledialog.askinteger("Threshold Value", "Enter the threshold value:", initialvalue=127)
    if threshold_value is None:
        return  # 사용자가 입력 대화 상자를 취소한 경우 아무 작업도 수행하지 않습니다.

    gray_image = grayscale(input_image, panelB)

    ret, thresh = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    thresh1 = Image.fromarray(thresh)
    thresh1 = ImageTk.PhotoImage(thresh1)

    image_label = Label(panelB, image=thresh1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = thresh1
    image_label.place(x=50, y=100)
 
    new_img=thresh

    return thresh

def redext(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    row,col,plane = input_image.shape

    red = np.zeros((row,col,plane),np.uint8)
    red[:,:,0] = input_image[:,:,0]

    red1 = Image.fromarray(red)

    red1= ImageTk.PhotoImage(red1)

    image_label = Label(panelB, image=red1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = red1
    image_label.place(x=50, y=100)

    new_img=red

    return red

def greenext(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    row,col,plane = input_image.shape

    green= np.zeros((row,col,plane),np.uint8)
    green[:,:,1] = input_image[:,:,1]

    green1 = Image.fromarray(green)

    green1= ImageTk.PhotoImage(green1)

    image_label = Label(panelB, image=green1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = green1
    image_label.place(x=50, y=100)

    new_img=green

    return green

def blueext(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    row,col,plane = input_image.shape

    blue = np.zeros((row,col,plane),np.uint8)
    blue[:,:,2] = input_image[:,:,2]

    blue1 = Image.fromarray(blue)

    blue1= ImageTk.PhotoImage(blue1)

    image_label = Label(panelB, image=blue1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = blue1
    image_label.place(x=50, y=100)
    new_img=blue

    return blue



def edge(input_image, panelB):   # image = threshold(input_image,panelB)
    global new_img, image_label

    thresholds = simpledialog.askstring("Canny Edge Detection", "Enter two threshold values separated by a space (e.g., '30 60'):")
    if thresholds is None:
        return  # 사용자가 입력 대화 상자를 취소한 경우 아무 작업도 수행하지 않습니다.

    low_threshold, high_threshold = map(int, thresholds.split())

    edged = cv2.Canny(input_image, low_threshold, high_threshold)
    edged1 = Image.fromarray(edged)
    edged1 = ImageTk.PhotoImage(edged1)

    image_label = Label(panelB, image=edged1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = edged1
    image_label.place(x=50, y=100)
    
    new_img=edged

    return edged

def skeleton(input_image, panelB):
    global new_img, image_label
    thr_image = threshold(input_image, panelB)
    skel = np.zeros(thr_image.shape, np.uint8)

    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

    while True:
        open = cv2.morphologyEx(thr_image, cv2.MORPH_OPEN, element)
        temp = cv2.subtract(thr_image, open)
        eroded = cv2.erode(thr_image, element)
        skel = cv2.bitwise_or(skel, temp)
        thr_image = eroded.copy()
        if cv2.countNonZero(thr_image) == 0:
            break

    skel1 = Image.fromarray(skel)
    skel1 = ImageTk.PhotoImage(skel1)

    if image_label is not None:
        image_label.destroy()

    image_label = Label(panelB, image=skel1, anchor='s')
    image_label.image = skel1
    image_label.place(x=50, y=100)
    new_img = skel

    return skel



def denoise(input_image, panelB):   # smoothening
    global new_img, image_label

    # Get visual_patch_size and h from user using simpledialog
    visual_patch_size = simpledialog.askinteger("Input", "Enter visual_patch_size:", minvalue=1)
    h = simpledialog.askfloat("Input", "Enter h:", minvalue=1)

    # Check if the user canceled the input
    if visual_patch_size is None or h is None:
        return None

    denoise = cv2.fastNlMeansDenoisingColored(input_image, None, int(visual_patch_size), 5, 7, int(h))
    denoise1 = Image.fromarray(denoise)

    denoise1 = ImageTk.PhotoImage(denoise1)

    image_label = Label(panelB, image=denoise1, anchor='s')  
    image_label.image = denoise1
    image_label.place(x=50, y=100)
    
    new_img = denoise

    return denoise




def sharp(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharpened = cv2.filter2D(input_image, ddepth=-1, kernel=kernel)

    sharpened1 = Image.fromarray(sharpened)

    sharpened1= ImageTk.PhotoImage(sharpened1)

    image_label = Label(panelB, image=sharpened1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = sharpened1
    image_label.place(x=50, y=100)
    new_img=sharpened

    return sharpened


def histo(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    histogram = cv2.cvtColor(input_image, cv2.COLOR_BGR2YUV)
    histogram [:,:,0] = cv2.equalizeHist(histogram [:,:,0])
    histogram = cv2.cvtColor(histogram, cv2.COLOR_YUV2BGR)
    histogram1 = Image.fromarray(histogram)

    histogram1= ImageTk.PhotoImage(histogram1)

    image_label = Label(panelB, image=histogram1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = histogram1
    image_label.place(x=50, y=100)
    new_img=histogram

    return histogram


def powerlawtrans(input_image, panelB):
    global new_img, image_label  # 여기에 image_label 추가
    
    # 사용자로부터 감마 값을 입력받음
    gamma_value = simpledialog.askfloat("Gamma Value", "Enter the gamma value:")
    
    # 감마 값에 따라 이미지 변환
    gammaplt = np.array(255 * (input_image / 255) ** gamma_value, dtype='uint8')
    gammaplt1 = Image.fromarray(gammaplt)

    gammaplt1 = ImageTk.PhotoImage(gammaplt1)

    # 이미지 라벨 생성
    image_label = Label(panelB, image=gammaplt1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = gammaplt1
    image_label.place(x=50, y=100)
    
    new_img = gammaplt

    return gammaplt


def maskimg(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    x, y , w, h = cv2.selectROI(image)
    start = (x, y)
    end = (x + w, y + h)
    rect = (x, y , w, h)

    cv2.rectangle(input_image, start, end, (0,0,255), 3)
    mask = np.zeros(input_image.shape[:2], np.uint8)

    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    cv2.grabCut(input_image, mask, rect, bgdModel, fgdModel, 100, cv2.GC_INIT_WITH_RECT)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask1 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    maskimage = input_image * mask1[:, :, np.newaxis]

    maskimage1 = Image.fromarray(maskimage)

    maskimage1= ImageTk.PhotoImage(maskimage1)

    image_label = Label(panelB, image=maskimage1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = maskimage1
    image_label.place(x=50, y=100)
    new_img=maskimage

    return maskimage

def pencil(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    img=grayscale(input_image,panelB)
    img_invert = cv2.bitwise_not(img)
    img_smoothing = cv2.GaussianBlur(img_invert, (25, 25),sigmaX=0, sigmaY=0)

    pencilimg = cv2.divide(img, 255 - img_smoothing, scale=255)
    pencilimg1= Image.fromarray(pencilimg)

    pencilimg1= ImageTk.PhotoImage(pencilimg1)

    image_label = Label(panelB, image=pencilimg1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = pencilimg1
    image_label.place(x=50, y=100)
    new_img=pencilimg

    return pencilimg

def colpencil(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    img_invert = cv2.bitwise_not(input_image)
    img_smoothing = cv2.GaussianBlur(img_invert, (21, 21),sigmaX=0, sigmaY=0)

    colpencilimg = cv2.divide(input_image, 255- img_smoothing, scale=255)
    colpencilimg1= Image.fromarray(colpencilimg)

    colpencilimg1= ImageTk.PhotoImage(colpencilimg1)

    image_label = Label(panelB, image=colpencilimg1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = colpencilimg1
    image_label.place(x=50, y=100)
    new_img=colpencilimg

    return colpencilimg

def cartoon(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    gray=grayscale(input_image,panelB)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)

    color = cv2.bilateralFilter(input_image, 9, 250, 250)
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    cartoon1= Image.fromarray(cartoon)

    cartoon1= ImageTk.PhotoImage(cartoon1)

    image_label = Label(panelB, image=cartoon1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = cartoon1
    image_label.place(x=50, y=100)
    new_img=cartoon

    return cartoon

def watercolor(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    watercolor = cv2.stylization(input_image, sigma_s=100, sigma_r=0.45)

    watercolor1= Image.fromarray(watercolor)

    watercolor1= ImageTk.PhotoImage(watercolor1)

    image_label = Label(panelB, image=watercolor1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = watercolor1
    image_label.place(x=50, y=100)
    new_img=watercolor

    return watercolor

def emboss(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가
    kernel = np.array([[0,-1,-1],[1,0,-1],[1,1,0]])
    emboss = cv2.filter2D(input_image, kernel=kernel, ddepth=-1)
    emboss= cv2.cvtColor(emboss, cv2.COLOR_BGR2GRAY)
    emboss=255-emboss
    emboss1= Image.fromarray(emboss)

    emboss1= ImageTk.PhotoImage(emboss1)

    image_label = Label(panelB, image=emboss1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = emboss1
    image_label.place(x=50, y=100)
    new_img=emboss

    return emboss

def move_image(panelA,panelB):
    global image,image_label  # 여기에 image_label 추가
    # Get the image from 'Panel B'
    img_to_move = image_label.image


    image_label_A = Label(panelA, image=img_to_move, anchor='s')  # 's'는 하단 정렬을 의미
    image_label_A.image = img_to_move
    image_label_A.place(x=50, y=100)
    
    image=new_img
    # Clear the image in 'Panel B' by setting an empty image
    blank_image = ImageTk.PhotoImage(Image.new('RGB', (500, 500)))
    

    image_label = Label(panelB, image=blank_image, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = blank_image
    image_label.place(x=50, y=100)
    
    return image

def apply_averaging(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    kernel = np.ones((5,5),np.float32)/25
    filtered_image = cv2.filter2D(input_image,-1,kernel)
    
    filtered_image1= Image.fromarray(filtered_image)

    filtered_image1= ImageTk.PhotoImage(filtered_image1)
    
    
    image_label = Label(panelB, image=filtered_image1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = filtered_image1
    image_label.place(x=50, y=100)

    new_img=filtered_image

    return filtered_image

def apply_gaussian(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    filtered_image = cv2.GaussianBlur(input_image,(5,5),0)
    filtered_image1= Image.fromarray(filtered_image)

    filtered_image1= ImageTk.PhotoImage(filtered_image1)
    
    
    image_label = Label(panelB, image=filtered_image1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = filtered_image1
    image_label.place(x=50, y=100)

    new_img=filtered_image

    return filtered_image
    
def apply_median(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    filtered_image = cv2.medianBlur(input_image, 5)
    filtered_image1= Image.fromarray(filtered_image)

    filtered_image1= ImageTk.PhotoImage(filtered_image1)
    
    
    image_label = Label(panelB, image=filtered_image1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = filtered_image1
    image_label.place(x=50, y=100)

    new_img=filtered_image

    return filtered_image


def apply_bilateral(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    filtered_image = cv2.bilateralFilter(input_image,9,75,75)
    filtered_image1= Image.fromarray(filtered_image)

    filtered_image1= ImageTk.PhotoImage(filtered_image1)
    
    
    image_label = Label(panelB, image=filtered_image1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = filtered_image1
    image_label.place(x=50, y=100)

    
    new_img=filtered_image

    return filtered_image


def sift_features(input_image, panelB):
    global new_img, image_label

    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints = sift.detect(gray, None)
    keypoints = sorted(keypoints, key=lambda x: -x.response)[:30]  # 상위 100개의 키포인트만 선택

    img_sift = cv2.drawKeypoints(gray, keypoints, input_image)

    img_sift1 = Image.fromarray(img_sift)
    img_sift1 = ImageTk.PhotoImage(img_sift1)

    image_label = Label(panelB, image=img_sift1, anchor='s')
    image_label.image = img_sift1
    image_label.place(x=50, y=100)

    new_img = img_sift

    return img_sift




# from tensorflow.keras.preprocessing.image import img_to_array

# def cnn_features(input_image, panelB):
#     global new_img, image_label

#     # Load the pre-trained VGG-16 model
#     model = VGG16(weights='imagenet', include_top=False)

#     # Resize and preprocess the image for the VGG-16 model
#     img = cv2.resize(input_image, (224, 224))  # Resize to 224x224 pixels
#     img = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

#     x = img_to_array(img)
#     x = np.expand_dims(x, axis=0)

#     x = preprocess_input(x)

#     features = model.predict(x)

#     return features



def pca_features(input_image, panelB):
    global new_img, image_label

    n_components = askinteger("PCA Component Count", "Enter the number of PCA components:", initialvalue=20)
    if n_components is None:
        return  # If the user cancels the input, do nothing

    blue, green, red = cv2.split(input_image)

    pca = PCA(n_components)

    # Applying PCA to the red channel and then applying inverse transform to the transformed array.
    red_transformed = pca.fit_transform(red)
    red_inverted = pca.inverse_transform(red_transformed)

    # Applying PCA to the green channel and then applying inverse transform to the transformed array.
    green_transformed = pca.fit_transform(green)
    green_inverted = pca.inverse_transform(green_transformed)

    # Applying PCA to the blue channel and then applying inverse transform to the transformed array.
    blue_transformed = pca.fit_transform(blue)
    blue_inverted = pca.inverse_transform(blue_transformed)

    image_pca_np = np.dstack((red_inverted, red_inverted, red_inverted))
    img_compressed = image_pca_np.astype(np.uint8)

    pil_image_pca = Image.fromarray(img_compressed)
    tk_image_pca = ImageTk.PhotoImage(pil_image_pca)

    image_label = Label(panelB, image=tk_image_pca, anchor='s')
    image_label.image = tk_image_pca
    image_label.place(x=50, y=100)

    new_img = image_pca_np

    return image_pca_np




def lbp_features(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Compute LBP features
    lbp = feature.local_binary_pattern(gray, P=24, R=8, method="uniform")

    # Normalize the LBP image (for better visualization)
    lbp = (lbp - lbp.min()) / (lbp.max() - lbp.min())

    img_lbp= Image.fromarray(lbp)
    img_lbp1= ImageTk.PhotoImage(img_lbp)

    image_label = Label(panelB, image=img_lbp1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = img_lbp1
    image_label.place(x=50, y=100)

    new_img=lbp

    return np.array(lpb)

def gabor_features(input_image,panelB):
    global new_img, image_label  # 여기에 image_label 추가

    # Gabor filter parameters
    ksize = 31     # gabor kernel size
    sigma = 1.5    # Sigma of the gaussian
    theta = np.pi/4   # Orientation angle
    lamda = np.pi/4   # Wavelength 
    gamma = 0.5   # Spatial aspect ratio.

    gray_image = cv2.cvtColor(input_image,cv2.COLOR_BGR2GRAY)  

    g_kernel = cv2.getGaborKernel((ksize,ksize), sigma, theta, lamda, gamma)

    filtered_img=cv2.filter2D(gray_image,cv2.CV_8UC3,g_kernel)

    filtered_img1= Image.fromarray(filtered_img)
    filtered_img1= ImageTk.PhotoImage(filtered_img1)

    image_label = Label(panelB, image=filtered_img1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = filtered_img1
    image_label.place(x=50, y=100)

    new_img = filtered_img

    return filtered_img




from torchvision.models.segmentation import FCN_ResNet101_Weights
from torchvision.models.segmentation import DeepLabV3_ResNet101_Weights


fcn_net = models.segmentation.fcn_resnet101(weights=FCN_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1).eval()
dlab = models.segmentation.deeplabv3_resnet101(weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1).eval()

def decode_segmap(image, nc=21):  
    label_colors = np.array([(i*10%255,i*20%255,i*30%255) for i in range(nc)]) 

    r,g,b=np.zeros_like(image),np.zeros_like(image),np.zeros_like(image)
    
    for l in range(1,nc):
        idx=image==l
        
        r[idx]=label_colors[l][2]
        g[idx]=label_colors[l][1]
        b[idx]=label_colors[l][2]

    rgb=np.stack([r,g,b],axis=-1)

    return rgb

def segment(net,input_image,panelB, show_orig=True, dev='cpu'):
    global new_img  # Add this line to declare 'panelB' as a global variable

    img = Image.fromarray(input_image)  # Use the global variable image
    if show_orig: plt.imshow(img); plt.axis('off'); plt.show() # show_orig=true면 이미지 보여주기
  
    # T.Resize(640)
    trf = T.Compose([T.Resize((500,500)), 
                     T.ToTensor(), 
                     T.Normalize(mean = [0.485, 0.456, 0.406], 
                                 std = [0.229, 0.224, 0.225])])
    
    inp = trf(img).unsqueeze(0).to(dev)
    out = net.to(dev)(inp)['out']
    
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    
    rgb_np=decode_segmap(om)
    semantic_tk=ImageTk.PhotoImage(image=Image.fromarray(rgb_np.astype('uint8'), 'RGB'))

    image_label = Label(panelB, image=semantic_tk, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = semantic_tk
    image_label.place(x=50, y=100)
    
    new_img=rgb_np
    return rgb_np





from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess_input, decode_predictions as vgg_decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess_input, decode_predictions as resnet_decode_predictions
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as efficientnet_preprocess_input, decode_predictions as efficientnet_decode_predictions
from tensorflow.keras.applications import DenseNet121, MobileNetV2, InceptionV3, Xception
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions



# ... (이미지 업로드 및 기타 함수들은 그대로 유지)

# Applying DL/ML Methods 서브메뉴에서 VGG16 모델을 사용하여 이미지 분류
def apply_vgg_classification():
    global image, panelB

    # VGG16 모델 불러오기
    vgg_model = VGG16(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (224, 224))  # VGG16 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = vgg_preprocess_input(img_array)

    predictions = vgg_model.predict(img_array)
    decoded_predictions = vgg_decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시
    result_label = tk.Label(panelB, text=f"VGG16 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=100)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")

# Applying DL/ML Methods 서브메뉴에서 ResNet50 모델을 사용하여 이미지 분류
def apply_resnet_classification():
    global image, panelB

    # ResNet50 모델 불러오기
    resnet_model = ResNet50(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (224, 224))  # ResNet50 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet_preprocess_input(img_array)

    predictions = resnet_model.predict(img_array)
    decoded_predictions = resnet_decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시
    result_label = tk.Label(panelB, text=f"ResNet50 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=180)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")
    
# Applying DL/ML Methods 서브메뉴에서 EfficientNetB0 모델을 사용하여 이미지 분류
def apply_efficientnet_classification():
    global image, panelB

    # EfficientNetB0 모델 불러오기
    efficientnet_model = EfficientNetB0(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (224, 224))  # EfficientNetB0 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = efficientnet_preprocess_input(img_array)

    predictions = efficientnet_model.predict(img_array)
    decoded_predictions = efficientnet_decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시
    result_label = tk.Label(panelB, text=f"EfficientNetB0 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=260)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")
    
    
def apply_densenet_classification():
    global image, panelB

    # DenseNet121 모델 불러오기
    densenet_model = DenseNet121(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (224, 224))  # DenseNet121 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = densenet_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시하면서 텍스트 위치, 크기, 테두리 설정
    result_label = tk.Label(panelB, text=f"DenseNet121 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=340)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")

def apply_mobilenetv2_classification():
    global image, panelB

    # MobileNetV2 모델 불러오기
    mobilenetv2_model = MobileNetV2(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (224, 224))  # MobileNetV2 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = mobilenetv2_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시하면서 텍스트 위치, 크기, 테두리 설정
    result_label = tk.Label(panelB, text=f"MobileNetV2 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=420)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")

def apply_inceptionv3_classification():
    global image, panelB

    # InceptionV3 모델 불러오기
    inceptionv3_model = InceptionV3(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (299, 299))  # InceptionV3 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = inceptionv3_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시하면서 텍스트 위치, 크기, 테두리 설정
    result_label = tk.Label(panelB, text=f"InceptionV3 Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=500)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")

def apply_xception_classification():
    global image, panelB

    # Xception 모델 불러오기
    xception_model = Xception(weights='imagenet')

    # 이미지 전처리 및 예측
    img_array = image.copy()
    img_array = cv2.resize(img_array, (299, 299))  # Xception 모델의 입력 크기에 맞게 조정
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = xception_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]

    # 결과를 panelB에 표시하면서 텍스트 위치, 크기, 테두리 설정
    result_label = tk.Label(panelB, text=f"Xception Prediction: {decoded_predictions[0][1]}, Confidence: {decoded_predictions[0][2]:.2%}")
    result_label.place(x=20, y=580)
    result_label.config(font=("Arial", 14))
    result_label.config(bd=2, relief="solid")
################################함수###############################



C:\Users\user\Anaconda3\envs\test01\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import threading
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
import random
import shutil



def rename_files_in_folder():
    # 사용자에게 폴더 선택하도록 하기
    folder_path = filedialog.askdirectory()  # 폴더 하나 선택
    if not folder_path:
        return
    # 폴더 이름을 라벨로 사용
    label = os.path.basename(folder_path)
    # 폴더 안의 파일 이름 바꾸기
    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # 이미지 파일인 경우
            dst = label + "_" + str(idx) + ".jpg"
            src = folder_path + '/' + filename
            dst = folder_path + '/' + dst
            os.rename(src, dst)  # 파일 이름 변경
            
            

def distribute_files():
    # 사용자에게 폴더 선택하도록 하기
    folder_path = filedialog.askdirectory()  # 폴더 하나 선택
    if not folder_path:
        return

    # 선택한 폴더의 상위 폴더 경로 얻기
    parent_folder = os.path.dirname(folder_path)

    # 상위 폴더에 Train_set, Test_set 폴더 생성
    train_folder = os.path.join(parent_folder, 'Train_set')
    test_folder = os.path.join(parent_folder, 'Test_set')
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # 이미지 파일 리스트 생성
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]
    random.shuffle(image_files)  # 무작위로 섞기

    # 이미지 파일 분배
    num_train = int(len(image_files) * 0.7)  # 70%를 훈련 세트로
    for i, f in enumerate(image_files):
        src = os.path.join(folder_path, f)
        # 훈련 세트에 복사
        if i < num_train:
            dst = os.path.join(train_folder, f)
        # 테스트 세트에 복사
        else:
            dst = os.path.join(test_folder, f)
        shutil.copy(src, dst)
        
        
def open_hyperparam_dialog(new_window):
    # 새 창 생성
    dialog = tk.Toplevel(new_window)

    # 각각의 선택 사항에 대한 변수 생성
    model_var = tk.StringVar(dialog)
    optimizer_var = tk.StringVar(dialog)
    lr_var = tk.StringVar(dialog)
    epoch_var = tk.StringVar(dialog)

    # 각 선택 항목의 라벨 생성 및 배치
    model_label = tk.Label(dialog, text='Model')
    optimizer_label = tk.Label(dialog, text='Optimizer')
    lr_label = tk.Label(dialog, text='Learning Rate')
    epoch_label = tk.Label(dialog, text='Epochs')
    


    # OptionMenu 위젯으로 각각의 선택 사항 표시
    model_option = tk.OptionMenu(dialog, model_var, 'VGG16', 'ResNet50', 'InceptionV3')
    optimizer_option = tk.OptionMenu(dialog, optimizer_var, 'Adam', 'SGD', 'RMSprop')
    lr_option = tk.OptionMenu(dialog, lr_var, '0.1', '0.01', '0.001', '0.0001')
    epoch_option = tk.OptionMenu(dialog, epoch_var, '1', '5', '10', '20')

    # 위젯 배치
    model_label.pack()
    model_option.pack()

    optimizer_label.pack()
    optimizer_option.pack()

    lr_label.pack()
    lr_option.pack()

    epoch_label.pack()
    epoch_option.pack()


  # 선택한 학습 파라미터를 저장하는 리스트
    selected_params = []

    # OK 버튼 생성 및 배치. 버튼을 누르면 선택한 값을 저장하고 창을 닫음
    ok_button = tk.Button(dialog, text='OK', command=lambda: selected_params.extend([model_var.get(), optimizer_var.get(), lr_var.get(), epoch_var.get()]) or dialog.destroy())
    ok_button.pack()

    # 창이 닫힐 때까지 대기
    dialog.wait_window()

    # 선택한 학습 파라미터 반환
    return selected_params
        

def get_model(model_name):
    if model_name == 'VGG16':
        return VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet50':
        return ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        return InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        raise ValueError("Invalid model name")
        
def get_optimizer(optimizer, learning_rate):
    if optimizer == 'Adam':
        return Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        return SGD(learning_rate=learning_rate)
    elif optimizer == 'RMSprop':
        return RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")
        

def train_model():
    model_name, optimizer, learning_rate, epochs = open_hyperparam_dialog(new_window)
    base_model = get_model(model_name)

    folder_path = filedialog.askdirectory()  # 폴더 선택
    if not folder_path:
        return

    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]

    X = []
    y = []
    for image_file in image_files:
        # 이미지 로드 및 전처리
        img = load_img(os.path.join(folder_path, image_file), target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        X.append(img_array)

        # 레이블 추출 및 인코딩
        label = image_file.split('_')[0]
        y.append(label)
    X = np.vstack(X)  # 배열들을 수직으로(열 방향으로) 쌓기

    # 레이블 인코딩
    le = LabelEncoder()
    y = le.fit_transform(y)
    y = to_categorical(y)

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 모델 수정
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(len(le.classes_), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # 모델 컴파일
    opt = get_optimizer(optimizer, float(learning_rate))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 학습
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=int(epochs))
    
    global trained_model
    trained_model=model



def test_model(trained_model):
    
    global y_test_gb, y_pred_classes_gb

    folder_path = filedialog.askdirectory()  # 폴더 선택
    if not folder_path:
        return

    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]

    X_test = []
    y_test_gb = []
    for image_file in image_files:
        # 이미지 로드 및 전처리
        img = load_img(os.path.join(folder_path, image_file), target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        X_test.append(img_array)

        # 레이블 추출 및 인코딩
        label = image_file.split('_')[0]
        y_test_gb.append(label)
    X_test = np.vstack(X_test)  # 배열들을 수직으로(열 방향으로) 쌓기
    
    
    # 레이블 인코딩
    le = LabelEncoder()
    y_test_gb = le.fit_transform(y_test_gb)
    
    model=trained_model
    # 모델 예측
    y_pred = model.predict(X_test)
    y_pred_classes_gb = np.argmax(y_pred, axis=1)
    

def visuallization(y_test_gb, y_pred_classes_gb,new_window):
    # 새 창 생성
    window = tk.Toplevel(new_window)
    canvas = tk.Canvas(window)
    scrollbar = tk.Scrollbar(window, command=canvas.yview)
    frame = tk.Frame(canvas)
    

    try:
        # 성능 지표 계산
        accuracy = accuracy_score(y_test_gb, y_pred_classes_gb)
        precision = precision_score(y_test_gb, y_pred_classes_gb, zero_division=0)
        recall = recall_score(y_test_gb, y_pred_classes_gb, zero_division=0)
        f1 = f1_score(y_test_gb, y_pred_classes_gb, zero_division=0)
        roc_auc = roc_auc_score(y_test_gb, y_pred_classes_gb)

        # 성능 지표 표시
        tk.Label(frame, text=f'Accuracy: {accuracy:.2f}').pack()
        tk.Label(frame, text=f'Precision: {precision:.2f}').pack()
        tk.Label(frame, text=f'Recall: {recall:.2f}').pack()
        tk.Label(frame, text=f'F1 Score: {f1:.2f}').pack()
        tk.Label(frame, text=f'ROC AUC: {roc_auc:.2f}').pack()

        # ROC Curve 생성
        fpr, tpr, _ = roc_curve(y_test_gb, y_pred_classes_gb)
        fig1 = Figure(figsize=(5, 5))
        a1 = fig1.add_subplot(111)
        a1.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        a1.plot([0, 1], [0, 1], 'k--')
        a1.set_xlim([0.0, 1.0])
        a1.set_ylim([0.0, 1.05])
        a1.set_xlabel('False Positive Rate')
        a1.set_ylabel('True Positive Rate')
        a1.set_title('Receiver Operating Characteristic')
        a1.legend(loc="lower right")
        # ROC Curve 표시
        canvas1 = FigureCanvasTkAgg(fig1, master=frame)
        canvas1.get_tk_widget().pack()
    except ValueError:
        tk.Label(frame, text='Only one class present in y_true or y_pred. Cannot calculate performance metrics.').pack()

    # Confusion Matrix 생성
    cm = confusion_matrix(y_test_gb, y_pred_classes_gb)
    fig2 = Figure(figsize=(5, 5))
    a2 = fig2.add_subplot(111)
    cax = a2.matshow(cm)
    fig2.colorbar(cax)

    # Confusion Matrix 표시
    canvas2 = FigureCanvasTkAgg(fig2, master=frame)
    canvas2.get_tk_widget().pack()

    # 스크롤 바 설정
    canvas.create_window((0, 0), window=frame, anchor='nw')
    frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox('all')))
    canvas.configure(yscrollcommand=scrollbar.set)

    # 위젯 배치
    canvas.pack(side='left', fill='both', expand=True)
    scrollbar.pack(side='right', fill='y')

    


    
from sklearn.cluster import KMeans

def k_means_clustering(input_image, panelB, k=3):
    global new_img, image_label

    # Reshape the image to a 2D array of pixels
    pixels = input_image.reshape((-1, 3))

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)

    # Assign each pixel to a cluster
    labels = kmeans.labels_

    # Replace each pixel with its corresponding centroid
    segmented_image = kmeans.cluster_centers_.astype(int)[labels]

    # Reshape the segmented image back to its original shape
    segmented_image = segmented_image.reshape(input_image.shape)

    # Convert to uint8
    segmented_image = segmented_image.astype(np.uint8)

    # Display the segmented image in panelB
    segmented_img = Image.fromarray(segmented_image)
    segmented_img = ImageTk.PhotoImage(segmented_img)

    image_label = Label(panelB, image=segmented_img, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = segmented_img
    image_label.place(x=50, y=100)

    new_img = segmented_image

    return segmented_image


from sklearn.cluster import MeanShift, estimate_bandwidth

image_label = None  # Declare image_label as a global variable
def mean_shift_clustering(input_image, panelB):
    global new_img, image_label

    # Reshape the image to a 2D array of pixels
    pixels = input_image.reshape((-1, 3))

    # Estimate bandwidth for mean shift
    bandwidth = estimate_bandwidth(pixels, quantile=0.2, n_samples=500)

    # Perform mean shift clustering
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(pixels)

    # Assign each pixel to a cluster
    labels = ms.labels_

    # Replace each pixel with its corresponding centroid
    segmented_image = ms.cluster_centers_.astype(int)[labels]

    # Reshape the segmented image back to its original shape
    segmented_image = segmented_image.reshape(input_image.shape)

    # Convert to uint8
    segmented_image = segmented_image.astype(np.uint8)

    # Display the segmented image in panelB
    segmented_img = Image.fromarray(segmented_image)
    segmented_img = ImageTk.PhotoImage(segmented_img)

    if image_label is not None:
        image_label.destroy()  # Remove previous image label in panelB

    image_label = Label(panelB, image=segmented_img, anchor='s')  # 's'는 하단 정렬을 의미
    image_label.image = segmented_img
    image_label.place(x=50, y=100)

    new_img = segmented_image

    return segmented_image
    
    
    

In [3]:

def create_menu_button(parent, label, command=None):
    button = tk.Menubutton(parent, text=label, relief="raised")
    menu = tk.Menu(button)
    button.config(menu=menu)
    menu.add_command(label=label, command=command)
    return button



def upload_image():
    global image, image_label_A, panelB
    f_types = [('PNG 파일', '*.png'), ('Jpg 파일', '*.jpg')]
    path = filedialog.askopenfilename(filetypes=f_types)
    
    image = cv2.imread(path)
    image = cv2.resize(image, (500, 500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image1 = Image.fromarray(image)
    image1 = ImageTk.PhotoImage(image1)

    # 새 이미지를 panelA에 표시
    image_label_A = tk.Label(panelA, image=image1, anchor='s')  # 's'는 하단 정렬을 의미
    image_label_A.image = image1
    image_label_A.place(x=50, y=100)
    
    panelB = tk.Frame(new_window, background='white', width=50, height=10)
    panelB.grid(row=0, column=2, sticky='nsew')

    return image
    


def clear_panels():
    global image_label_A, image_label, image, panelA, panelB   # 이미지 레이블 변수를 가져옴
    if image_label_A is not None:
        image_label_A.destroy()  # panelA에 있는 이미지 레이블 제거
    if image_label is not None:
        image_label.destroy()  # panelB에 있는 이미지 레이블 제거
    # panelA와 panelB를 완전히 초기화하려면 새로운 빈 프레임을 생성
    panelA = tk.Frame(new_window, background='white', width=50, height=10)
    panelB = tk.Frame(new_window, background='white', width=50, height=10)
    panelA.grid(row=0, column=1, sticky='nsew')
    panelB.grid(row=0, column=2, sticky='nsew')
    panelA.pack_propagate(0)  # panelA의 크기를 고정


def show_analysis():
    global new_window, image_label_A
    root.destroy()

    new_window = tk.Tk()

    panelS = tk.Frame(new_window, background='#DC143C', width=10, height=10)
    global panelA,panelB
    panelA = tk.Frame(new_window, background='white',width=50, height=10)
    panelB = tk.Frame(new_window,  background='white',width=50, height=10)

    panelS.grid(row=0, column=0, sticky='nsew')
    panelA.grid(row=0, column=1, sticky='nsew')
    panelB.grid(row=0, column=2, sticky='nsew')

    panelA.pack_propagate(0)  # panelA의 크기를 고정

    create_menu(new_window)

    new_window.grid_rowconfigure(0, weight=1)
    new_window.grid_columnconfigure(0, weight=1)
    new_window.grid_columnconfigure(1, weight=15)
    new_window.grid_columnconfigure(2, weight=15)
    

    button_labels = ['Inputting Image Data',  'Preprocessing','Filtering','Correction','Transformation', 'Feature Extraction',
                     'Segmentation', 'Clustering',
                     'Transfer Learning', 'Training Model', 'Updating an Image', 'Clear Panels']

    
# panelS의 가로 폭 설정
    panelS.grid_columnconfigure(0, weight=1)

    for i, name in enumerate(button_labels):
    # Create a Menubutton
        mbt = tk.Menubutton(panelS, text=name, height=2, font=("Arial", 13,"bold"), relief='raised')
        mbt.grid(row=i, sticky="ew")  # sticky 옵션을 사용하여 가로 크기를 패널의 가로 폭에 맞게 설정
        
        
        if name == 'Inputting Image Data':
            mbt.configure(highlightthickness=1, highlightbackground="crimson", highlightcolor="crimson",font=('Arial',12,"italic"))
        elif name == 'Updating an Image':
            mbt.configure(highlightthickness=1, highlightbackground="crimson", highlightcolor="crimson",font=('Arial',12,"italic"))
        elif name == 'Clear Panels':
            mbt.configure(highlightthickness=1, highlightbackground="crimson", highlightcolor="crimson",font=('Arial',12,"italic"))
        else:
            mbt.configure(highlightthickness=1, highlightbackground="crimson", highlightcolor="crimson")
 
        # Create a Menu for the Menubutton
        mnu = tk.Menu(mbt)


        if name == 'Inputting Image Data':
            mnu.add_command(label='Upload', command=upload_image)


        elif name == 'Preprocessing':
            mnu.add_command(label='Grayscale', command=lambda: grayscale(image, panelB))
            mnu.add_command(label='Invert Color', command=lambda: negative(image, panelB))
            mnu.add_command(label='Red Attribute', command=lambda: redext(image, panelB))
            mnu.add_command(label='Green Attribute', command=lambda: greenext(image, panelB))
            mnu.add_command(label='Blue Attribute', command=lambda: blueext(image, panelB))
            
            
        elif name == 'Filtering':
            mnu.add_command(label='Averaging Blur', command=lambda: apply_averaging(image, panelB))
            mnu.add_command(label='Gaussian Blur', command=lambda: apply_gaussian(image, panelB))
            mnu.add_command(label='Median Blur', command=lambda: apply_median(image, panelB))
            mnu.add_command(label='Bilateral Blur', command=lambda: apply_bilateral(image, panelB))
            mnu.add_command(label='Sharpening', command=lambda: sharp(image, panelB))
            mnu.add_command(label='Smoothening', command=lambda: denoise(image, panelB))


        elif name == 'Correction':
            mnu.add_command(label='Binary', command=lambda: threshold(image, panelB))
            mnu.add_command(label='Edge Detection', command=lambda: edge(image, panelB))
            mnu.add_command(label='Skeleton', command=lambda: skeleton(image, panelB))
            mnu.add_command(label='Power Law Transformation', command=lambda: powerlawtrans(image, panelB))
            mnu.add_command(label='Contrast Enhancement', command=lambda: histo(image, panelB))



        elif name == 'Transformation':
            mnu.add_command(label='Pencil Sketch', command=lambda: pencil(image, panelB))
            mnu.add_command(label='Color Pencil Sketch', command=lambda: colpencil(image, panelB))
            mnu.add_command(label='Cartoonify', command=lambda: cartoon(image, panelB))
            mnu.add_command(label='Watercolor', command=lambda: watercolor(image, panelB))
            mnu.add_command(label='Emboss', command=lambda: emboss(image, panelB))

        elif name == 'Feature Extraction':
            mnu.add_command(label='SIFT', command=lambda:sift_features(image,panelB))
            #mnu.add_command(label='**CNN(VGG16)', command=lambda:cnn_features(image,panelB))
            mnu.add_command(label='PCA', command=lambda:pca_features(image,panelB))
            ##mnu.add_command(label='**LBP', command=lambda:lbp_features(image,panelB))
            mnu.add_command(label='Gabor', command=lambda:gabor_features(image,panelB))

            
        elif name == 'Segmentation':
            mnu.add_command(label='Semantic(FCN)', command=lambda: segment(fcn_net,image,panelB))
            mnu.add_command(label='Semantic(DLAB)', command=lambda: segment(dlab,image,panelB))
            
            
        elif name =='Transfer Learning':   
            mnu.add_command(label='VGG16', command=apply_vgg_classification)
            mnu.add_command(label='ResNet50', command=apply_resnet_classification)
            mnu.add_command(label='EfficientNetB0', command=apply_efficientnet_classification)
            mnu.add_command(label='DenseNet121', command=apply_densenet_classification)
            mnu.add_command(label='MobileNetV2', command=apply_mobilenetv2_classification)
            mnu.add_command(label='InceptionV3', command=apply_inceptionv3_classification)
            mnu.add_command(label='Xception', command=apply_xception_classification)
            
        elif name == 'Clustering':
            mnu.add_command(label='K-means clustering', command=lambda: k_means_clustering(image,panelB))
            mnu.add_command(label='mean shift clustering', command=lambda: mean_shift_clustering(image,panelB))

            
        elif name =='Updating an Image':
            mnu.add_command(label='Apply Image to Source Image', command=lambda:move_image(panelA,panelB))
        
        elif name == 'Clear Panels':
            mnu.add_command(label='Clear Panels', command=clear_panels)
            
        elif name == 'Training Model':
            mnu.add_command(label='1.Rename Images', command=rename_files_in_folder)
            mnu.add_command(label='2.Make Train/Test Files', command=distribute_files)
            mnu.add_command(label='3.Train Model', command=train_model)
            mnu.add_command(label='4.Test Model', command=lambda:test_model(trained_model))
            mnu.add_command(label='5.Result', command=lambda:visuallization(y_test_gb, y_pred_classes_gb,new_window))
    



        # Attach the menu to the Menubutton

        else:
            for j in range(5):  # Add five options to the menu as an example.
                mnu.add_command(label=f'Sub Button {j+1}')

        # Attach the menu to the Menubutton
        mbt.configure(menu=mnu)


    
def create_menu(parent):
    parent.title('Analysis')

    menubar = tk.Menu(parent)

    parent.geometry("1800x900")

    filemenu = tk.Menu(menubar, tearoff=0)
    filemenu.add_command(label="Open")
    filemenu.add_command(label="Save")
    menubar.add_cascade(label="File", menu=filemenu)

    editmenu = tk.Menu(menubar, tearoff=0)
    editmenu.add_command(label="Undo")
    menubar.add_cascade(label="Edit", menu=editmenu)

    viewmenu = tk.Menu(menubar, tearoff=0)
    viewmenu.add_command(label="Zoom In")
    menubar.add_cascade(label="View", menu=viewmenu)

    option_menu = tk.Menu(menubar)
    option_menu.add_command(label='Settings')
    menubar.add_cascade(menu=option_menu, label='Options')

    help_menu = tk.Menu(menubar)
    menubar.add_cascade(menu=help_menu, label='Help')

    new_window.config(menu=menubar)


    
def show_main():
    analysis_button = create_button(root,"ANALYSIS", 1) 
    analysis_button.config(command=show_analysis,
                           fg="white",
                           width=15,
                           bg="crimson",
                           activebackground="crimson",
                           font=("Helvetica", 17, "bold"))

    
    cds_lab_button = create_button(root,"CDS LAB", 2)

    model_button = create_button(root,"MODEL", 3)

    function_button = create_button(root,"FUNCTION", 4)
    
    
    
def create_button(parent, text, row):
    button = tk.Button(parent, text=text, width=25, fg="crimson", bg="white", activebackground="crimson", font=("Helvetica", 10, "bold"))
    button.grid(row=row, column=0)
    return button


root = tk.Tk()
root.title('Cloud Alpha')


# 창의 시작 크기 설정 (예: 300x200)
root.geometry("800x600")

# 화면 해상도
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# update_idletasks() 함수를 호출하여 변경 사항을 즉시 반영
root.update_idletasks()

# Tkinter 창의 실제 크기
window_width = root.winfo_width()
window_height = root.winfo_height()

# 중앙에 위치시킬 때의 좌표 계산
position_top = int(screen_height / 2 - window_height / 2)
position_right = int(screen_width / 2 - window_width / 2)

# 창의 위치 설정
root.geometry("+{}+{}".format(position_right, position_top))


# 전역 변수로 선언
img = None
imgtk = None

try:
   # 이미지 로드 (여기서는 example.png라는 이름의 파일을 사용합니다.)
    img = Image.open("C:\\Users\\user\\Desktop\\project\\image\\korea.png") 
    img = img.resize((250, 250)) # 이미지 크기 조정
    imgtk = ImageTk.PhotoImage(img)

    # grid 설정
    root.grid_rowconfigure(0, weight=1)
    root.grid_rowconfigure(1, weight=3)  # 비율을 조정하여 이미지를 상단으로 이동시킵니다.
    root.grid_columnconfigure(0, weight=1)

    # 레이블 위젯 생성 후 이미지 설정
    label_img = tk.Label(root, image=imgtk)
    label_img.grid(row=0, column=0)
    
    
except FileNotFoundError: 
    print("The specified image file was not found.") 
    
show_main()


root.mainloop()

The specified image file was not found.
